<a href="https://colab.research.google.com/github/avinashronanki/hyperparameter_tuning/blob/master/Ray_Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install -U ray 

Requirement already up-to-date: ray in /usr/local/lib/python3.6/dist-packages (1.0.0)


In [1]:
!pip install "ray[tune]" transformers datasets

     |████████████████████████████████| 22.9MB 1.4MB/s 
     |████████████████████████████████| 1.3MB 40.9MB/s 
     |████████████████████████████████| 153kB 42.9MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 204kB 44.4MB/s 
     |████████████████████████████████| 81kB 1.4MB/s 
     |████████████████████████████████| 71kB 6.8MB/s 
     |████████████████████████████████| 1.3MB 38.7MB/s 
     |████████████████████████████████| 2.9MB 34.4MB/s 
     |████████████████████████████████| 133kB 41.7MB/s 
     |████████████████████████████████| 317kB 47.5MB/s 
     |████████████████████████████████| 2.9MB 41.6MB/s 
     |████████████████████████████████| 1.1MB 42.0MB/s 
     |████████████████████████████████| 890kB 41.1MB/s 
     |████████████████████████████████| 17.7MB 77kB/s 
     |████████████████████████████████| 245kB 47.4MB/s 
     |████████████████████████████████| 61kB 7.2MB/s 
     |████████████████████████████████| 296kB 44.7MB/s 
 

In [4]:
!pip install -U hyperopt

     |████████████████████████████████| 972kB 2.7MB/s 
  Found existing installation: hyperopt 0.1.2
    Uninstalling hyperopt-0.1.2:
      Successfully uninstalled hyperopt-0.1.2


In [15]:
from datasets import load_dataset, load_metric
from transformers import (AutoModelForSequenceClassification, AutoTokenizer,
                          Trainer, TrainingArguments)
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune.schedulers import AsyncHyperBandScheduler
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
dataset = load_dataset('glue', 'mrpc')
metric = load_metric('glue', 'mrpc')

def encode(examples):
    outputs = tokenizer(
        examples['sentence1'], examples['sentence2'], truncation=True)
    return outputs

encoded_dataset = dataset.map(encode, batched=True)

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        'distilbert-base-uncased', return_dict=True)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Evaluate during training and a bit more often
# than the default to be able to prune bad trials early.
# Disabling tqdm is a matter of preference.
training_args = TrainingArguments(
    "test", evaluate_during_training=True, eval_steps=500, disable_tqdm=True)
# trainer = Trainer(
#     args=training_args,
#     tokenizer=tokenizer,
#     train_dataset=encoded_dataset["train"],
#     eval_dataset=encoded_dataset["validation"],
#     model_init=model_init,
#     compute_metrics=compute_metrics,
# )

# # Defaut objective is the sum of all metrics
# # when metrics are provided, so we have to maximize it.
# trainer.hyperparameter_search(
#     direction="maximize", 
#     backend="ray", 
#     n_samples=10, # number of trials
#     # n_jobs=2  # number of parallel jobs, if multiple GPUs
# )

trainer = Trainer(
    args=training_args,
    tokenizer=tokenizer,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    model_init=model_init,
    compute_metrics=compute_metrics,
)

best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="ray",
    # Choose among many libraries:
    # https://docs.ray.io/en/latest/tune/api_docs/suggestion.html
    search_alg=HyperOptSearch())
    # Choose among schedulers:
    # https://docs.ray.io/en/latest/tune/api_docs/schedulers.html
    #scheduler=AsyncHyperBand())

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4)
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/7c99657241149a24692c402a5c3f34d4c9f1df5ac2e4c3759fadea38f6cb29c4/cache-1a6e71c763609a3b.arrow


KeyboardInterrupt: ignored

In [12]:
dataset

DatasetDict({'train': Dataset(features: {'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}, num_rows: 3668), 'validation': Dataset(features: {'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}, num_rows: 408), 'test': Dataset(features: {'sentence1': Value(dtype='string', id=None), 'sentence2': Value(dtype='string', id=None), 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], names_file=None, id=None), 'idx': Value(dtype='int32', id=None)}, num_rows: 1725)})